In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
torch.manual_seed(1234)
# Define the PINN architecture
class PINN(nn.Module):
    def __init__(self):
        super(PINN, self).__init__()
        
        self.hidden_layers = nn.Sequential(
            nn.Linear(2, 50), nn.ReLU(), #first hidden layer 
            nn.Linear(50, 50), nn.ReLU(),
            nn.Linear(50, 50), nn.ReLU(),
            nn.Linear(50, 50), nn.ReLU(),
            nn.Linear(50, 1)  # last hidden layer (option price)
        )

    def forward(self, x):
        return self.hidden_layers(x)

# Create an instance of the model
model = PINN()


In [2]:
def black_scholes_loss(model, X):
    """ Compute loss from Black-Scholes PDE """
    X.requires_grad = True  # Enable automatic differentiation
    u_pred = model(X)  # Predict option prices

    # Compute first derivatives
    grads = torch.autograd.grad(u_pred.sum(), X, create_graph=True)[0]
    u_t, u_s = grads[:, 0:1], grads[:, 1:2]

    # Compute second derivative
    grads2 = torch.autograd.grad(u_s.sum(), X, create_graph=True)[0]
    u_ss = grads2[:, 1:2]

    # Define Black-Scholes parameters
    sigma = 0.4  # Volatility
    r = 0.03     # Risk-free rate

    # Compute residual f(t, S)
    f_pred = u_t + 0.5 * sigma**2 * X[:, 1:2]**2 * u_ss + (r * X[:, 1:2] * u_s) - r * u_pred
    loss = torch.mean(f_pred**2)  # MSE loss

    return loss

In [3]:
import numpy as np

# Generate collocation points
N_f = 10000  # Number of collocation points
t_samples = torch.rand(N_f, 1) * 3  # Random time values (0 to T=3 years)
S_samples = torch.rand(N_f, 1) * 500  # Random stock prices (0 to $500)
X_f = torch.cat((t_samples, S_samples), dim=1)  # Stack into (t, S) pairs


In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.005)  # Learning rate = 0.005
epochs = 500

for epoch in range(epochs):
    optimizer.zero_grad()  # Reset gradients
    loss = black_scholes_loss(model, X_f)  # Compute PDE loss
    loss.backward()  # Backpropagation
    optimizer.step()  # Update weights

    if epoch % 50 == 0:
        print(f"Epoch {epoch}: Loss = {loss.item():.6f}")


Epoch 0: Loss = 0.000452
Epoch 50: Loss = 0.000000
Epoch 100: Loss = 0.000000
Epoch 150: Loss = 0.000000
Epoch 200: Loss = 0.000000
Epoch 250: Loss = 0.000000
Epoch 300: Loss = 0.000000


In [ ]:
import matplotlib.pyplot as plt

# Generate test data
S_test = torch.linspace(0, 500, 100).reshape(-1, 1)  # Asset prices
t_test = torch.full_like(S_test, 1.5)  # Midway through expiration

X_test = torch.cat((t_test, S_test), dim=1)
C_pred = model(X_test).detach().numpy()

# Plot the option price as a function of stock price
plt.figure(figsize=(15,8))
plt.plot(S_test.numpy(), C_pred, label="PINN Prediction")
plt.scatter(S_test,C_pred,s=10,marker='*',color='red',label='True Labels')
plt.xlabel("Stock Price")
plt.ylabel("Option Price")
plt.title("American Call Option Pricing using PINNs")
plt.legend()
plt.grid()
plt.show()